In [8]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.ops import MLP
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100, CIFAR10, MNIST
import copy
from tqdm import tqdm
from collections import OrderedDict
import random
import math
import matplotlib.pyplot as plt
from client_selector import ClientSelector
from data_splitter import DataSplitter

In [9]:
!pip install wandb -qU
import wandb
wandb.login()

wandb: Currently logged in as: ivanludvig (ivanludvigdev). Use `wandb login --relogin` to force relogin


True

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data

In [11]:
K = 100

params = {
    'K': K,
    'C': 0.1,
    'B': 64,
    'J': 4,
    # 'lr_server': 1e-1,
    'lr_client': 1e-1,
    'method': 'fedavg',
    'tau': 1e-3,
    # 'gamma': 5,
    'participation': 'uniform',
    'rounds': 2000
}

In [12]:
preprocess = transforms.Compose([
    transforms.RandomCrop((28, 28)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

train_dataset = CIFAR100('datasets/cifar100', train=True, transform=preprocess, download=True)
test_dataset = CIFAR100('datasets/cifar100', train=False, transform=preprocess, download=True)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
data_split_params = {
    'K': K,
    'split_method': 'non-iid',
    'n_labels': 5
}

data_splitter = DataSplitter(data_split_params, train_dataset)
client_datasets = data_splitter.split()

samples_per_client: 500
samples_per_label: 100


100%|██████████| 100/100 [00:01<00:00, 57.31it/s]


In [14]:
def split_train_test(d):
  train_size = int(0.8 * len(d))
  test_size = len(d) - train_size
  train_dataset, test_dataset = torch.utils.data.random_split(d, [train_size, test_size])

  return train_dataset, test_dataset

train_test_client_datasets = [split_train_test(d) for d in client_datasets]
train_client_datasets = [d[0] for d in train_test_client_datasets]
test_client_datasets = [d[1] for d in train_test_client_datasets]

In [15]:
client_selector = ClientSelector(params)

## Model

In [16]:
class LeNet5_circa(nn.Module):
    def __init__(self):
        super( LeNet5_circa, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(4 * 4 * 64, 384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, 100)

    def forward(self, x):
        x = self.pool(self.conv1(x).relu())
        x = self.pool(self.conv2(x).relu())
        x = torch.flatten(x, 1)
        x = self.fc1(x).relu()
        x = self.fc2(x).relu()
        x = self.fc3(x)

        return x


model = LeNet5_circa().cuda()
model.to('cuda')

criterion = torch.nn.CrossEntropyLoss().cuda()

In [17]:
wandb.init(
    project='fl',
    name=f'fed mod {data_split_params["split_method"]}, J={params["J"]}, lr={params["lr_client"]}, n_labels={data_split_params["n_labels"]}',
    config={**params, **data_split_params}
)

In [18]:
path = lambda t: f'/content/drive/My Drive/fl/{data_split_params["split_method"]}-mod-J{params["J"]}-lr{params["lr_client"]}-n_labels{data_split_params["n_labels"]}-{t}.pt'

backup = 0
if backup:
    model.load_state_dict(torch.load(path(backup)))
model

LeNet5_circa(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=384, bias=True)
  (fc2): Linear(in_features=384, out_features=192, bias=True)
  (fc3): Linear(in_features=192, out_features=100, bias=True)
)

## Utils

In [19]:
def reduce_w(w_list, f):
    return OrderedDict([
            (key, f([x[key] for x in w_list])) for key in w_list[0].keys()
        ])


def tensor_sum(tensors_list, weights=None):
    if weights:
      return torch.sum(torch.stack([t*w for t, w in zip(tensors_list, weights)]), dim=0)
    return torch.sum(torch.stack(tensors_list), dim=0)


def w_norm2(w):
    res = 0
    for key in w.keys():
        res += torch.linalg.vector_norm(w[key]) ** 2
    return math.sqrt(res)


def fed_adagrad(v, delta, params):
    delta_norm2 = w_norm2(delta)
    return v + delta_norm2


def fed_yogi(v, delta, params):
    delta_norm2 = w_norm2(delta)
    return v - (1-params['beta2']) * delta_norm2 * torch.sign(v - delta_norm2)


def fed_adam(v, delta, params):
    delta_norm2 = w_norm2(delta)
    return params['beta2'] * v + (1-params['beta2']) * delta_norm2


methods = {
    'adagrad': fed_adagrad,
    'yogi': fed_yogi,
    'adam': fed_adam
}

## Training

In [20]:
T = params['rounds']
test_freq = 50
save_freq = 200

In [21]:
def test(model, loader, verbose=False):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    test_loss = test_loss / len(loader)
    test_accuracy = 100. * correct / total
    if verbose:
        print(f'Test Loss: {test_loss:.6f} Acc: {test_accuracy:.2f}%')
    return test_accuracy, test_loss


def client_update(model, k, params):
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=params['lr_client'], weight_decay=4e-4)
    loader = DataLoader(train_client_datasets[k], batch_size=params['B'], shuffle=True)
    test_client_loader = DataLoader(test_client_datasets[k], batch_size=params['B'], shuffle=True)

    i = 0
    for i in range(params['J']):
        for batch_idx, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            i += 1

            if i >= params['J']:
              return model.state_dict(), *test(model, test_client_loader)

    return model.state_dict(), *test(model, test_client_loader)


def train(model, params):
    accuracies = []
    losses = []
    v = params['tau'] ** 2
    w = model.state_dict()
    m = reduce_w([w], lambda x: torch.mul(x[0], 0.0))
    for t in tqdm(range(T)):
        s = client_selector.sample()

        w_clients = []
        w_losses = []
        for k in s:
            client_w, client_acc, client_loss = client_update(copy.deepcopy(model), k, params)
            w_clients.append(client_w)
            w_losses.append(client_loss)

        w_sum = sum(w_losses)
        weights = [l/w_sum for l in w_losses]
        if params['method'] == 'fedavg':
            w = reduce_w(
                w_clients,
                lambda x: tensor_sum(x, weights)
            )
        else:
            deltas = [
                reduce_w(
                    [w, w_client],
                    lambda x: x[1] - x[0]
                ) for w_client in w_clients
            ]

            delta = reduce_w(
                deltas,
                lambda x: tensor_sum(x) / len(deltas)
            )

            m = reduce_w(
                [m, delta],
                lambda x: params['beta1'] * x[0] + (1-params['beta1']) * x[1]
            )

            v = methods[params['method']](v, delta, params)
            w = reduce_w(
                [w, m],
                lambda x: x[0] + params['lr_server'] * x[1] / (math.sqrt(v) + params['tau'])
            )

        model.load_state_dict(w)

        if t % test_freq == 0 or t == T-1:
            acc, loss = test(model, test_loader, True)
            accuracies.append(acc)
            losses.append(loss)
            wandb.log({'acc': acc, 'loss': loss, 'round': t})

        if t % save_freq == 0 or t == T-1:
            torch.save(model.state_dict(), path(t))

    return accuracies, losses


accuracies, losses = train(model, params)

  0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
  0%|          | 1/2000 [00:05<3:09:47,  5.70s/it]

Test Loss: 4.605740 Acc: 1.27%


  3%|▎         | 51/2000 [00:20<1:05:48,  2.03s/it]

Test Loss: 4.592994 Acc: 1.89%


  5%|▌         | 101/2000 [00:33<52:28,  1.66s/it]

...

Test Loss: 2.908374 Acc: 28.10%


 98%|█████████▊| 1951/2000 [08:39<01:04,  1.32s/it]

Test Loss: 2.887194 Acc: 28.80%


100%|██████████| 2000/2000 [08:52<00:00,  3.76it/s]

Test Loss: 2.925799 Acc: 28.24%
